<a href="https://colab.research.google.com/github/fathima-shabnam/Udemy-Data-Courses/blob/main/Udemy_courses_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Udemy data collection using udemy API**


# **Objective**

Objective of this project is to collect data from udemy courses related to "data" and anayze and visualize the dataset.

This project includes 2 steps
*   Data collection using Udemy API
*   Analysis and visualization in Tableau


In [1]:
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import json
import warnings
warnings.filterwarnings("ignore")

## Authenication for API

Request for API ClientID and Client Secret from Udemy from below link <br>
https://www.udemy.com/user/edit-api-clients/

Substitute ClientID and ClientSerect in the below code


In [2]:
auth_ = HTTPBasicAuth('drWYq6jonLQZhiPuwH0GLMbc0GYtBRhYdC30elgv', 
                      '8w0OH1yLgByZL2UhDN5DTlNbPBsJaVo15eryAqWqSoKacdmTGx5MKbWHYKsZr8x8rQm2OvFQFNjljT33SFn70kxcTFpO3wFxvuHLv4RMQbx6A43tF5RPPNsM7ay7fTeD')

## Data Retrieval using API

Documentation for API is the below link <br>
https://www.udemy.com/developers/affiliate/

Date of collection - 19th December 2022

In [3]:
# retrive courses from every page until the next variable in the response is null
url_ = "https://www.udemy.com/api-2.0/courses/?fields[course]=@all&page=1&search=data"
response = requests.get(url = url_,auth=auth_)
course_details_all = pd.json_normalize(json.loads(response.text)['results'])
more_pages = True
while more_pages:
    if json.loads(response.text)['next']  is None:
        more_pages = False
    else :
          url_ = json.loads(response.text)['next']
          response = requests.get(url = url_,auth=auth_)
          course_details_all = course_details_all.append(pd.json_normalize(json.loads(response.text)['results']))

In [4]:
course_details_all.shape

(10000, 287)

## Selecting columns needed for analysis

In [5]:
# keep only columns relavant for analysis
course_details_df = course_details_all[['_class','rating','num_practice_tests','type_label','discount.campaign.end_time','id','num_reviews',
           'num_published_practice_tests','published_time','discount.campaign.start_time','title','num_reviews_recent',
           'status_label','content_length_video','discount.code','is_paid','rating_distribution','has_certificate',
           'num_coding_exercises','primary_category.title_cleaned','price','num_quizzes','has_closed_caption',
           'num_assignments','primary_subcategory.title_cleaned','visible_instructors','num_lectures','created',
           'has_sufficient_preview_length','context_info.label.display_name','is_practice_test_course','num_published_lectures'
           ,'discount.price.amount','published_title','num_published_quizzes',
           'discount.list_price.amount','num_subscribers','num_curriculum_items',
           'discount.list_price.currency','avg_rating_recent','num_of_published_curriculum_objects','faq','discount.campaign.code','instructional_level_simple' ]]

## Adding additional columns

In [6]:
# Add some additional columns
course_details_df['instructor_count'] = course_details_df['visible_instructors'].apply(lambda x: 0 if x is None else len(x))
course_details_df['rating_1'] = course_details_df['rating_distribution'].apply(lambda x: x[0]['count'] )
course_details_df['rating_2'] = course_details_df['rating_distribution'].apply(lambda x: x[1]['count'] )
course_details_df['rating_3'] = course_details_df['rating_distribution'].apply(lambda x: x[2]['count'] )
course_details_df['rating_4'] = course_details_df['rating_distribution'].apply(lambda x: x[3]['count'] )
course_details_df['rating_5'] = course_details_df['rating_distribution'].apply(lambda x: x[4]['count'] )

In [9]:
#saving file 
course_details_df.to_csv("udemy_data_courses.csv",index = False)

## Next Steps

Analysis and Visualization on Tableau